In [39]:
import geopy.geocoders
from geopy.geocoders import Nominatim

geopy.geocoders.options.default_user_agent = 'my_app/2'
geopy.geocoders.options.default_timeout = None

geolocator = Nominatim(user_agent="skilled_nursing")

In [38]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

In [40]:
import sqlalchemy

In [41]:
import json
import re

In [27]:
from sodapy import Socrata

MyAppToken = '5YNrrmFobthjbfVSPQWEZWkkk'
username = 'jennifer.faith16@gmail.com'
password = 'Nashua1998!'

# Connect to dataset
client = Socrata("data.cms.gov",
                 MyAppToken,
                 username,
                 password)

results = client.get("s2uc-8wxp", limit = 1000000)

In [42]:
results

[{'week_ending': '2020-05-24T00:00:00.000',
  'federal_provider_number': '045277',
  'provider_name': 'ARKANSAS CONVALESCENT CENTER',
  'provider_address': '6301 SOUTH  HAZEL STREET',
  'provider_city': 'PINE BLUFF',
  'provider_state': 'AR',
  'provider_zip_code': '71603',
  'submitted_data': 'N',
  'county': 'Jefferson',
  'reporting_interval': 'Week 01 - May 24'},
 {'week_ending': '2020-05-24T00:00:00.000',
  'federal_provider_number': '055516',
  'provider_name': 'WINDSOR CHICO CARE CENTER',
  'provider_address': '188 COHASSET LANE',
  'provider_city': 'CHICO',
  'provider_state': 'CA',
  'provider_zip_code': '95926',
  'submitted_data': 'N',
  'county': 'Butte',
  'geolocation': {'type': 'Point', 'coordinates': [-121.852568, 39.750127]},
  'reporting_interval': 'Week 01 - May 24',
  ':@computed_region_dz63_q6gx': '6',
  ':@computed_region_ia25_mrsk': '219',
  ':@computed_region_p3pw_njwh': '8',
  ':@computed_region_x9pa_s2gq': '1183',
  ':@computed_region_vbss_8z7g': '638'},
 {'we

In [43]:
# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)

In [44]:
small = df[['federal_provider_number', 'provider_name', 'provider_address',
    'provider_city', 'provider_state', 'provider_zip_code', 'county', 'geolocation']]

In [45]:
small['full_address']=small['provider_address'] + ', ' + small['provider_city'] + ', ' + small['provider_state'] + ', ' +  small['provider_zip_code']

<ipython-input-45-7a40433a1853>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small['full_address']=small['provider_address'] + ', ' + small['provider_city'] + ', ' + small['provider_state'] + ', ' +  small['provider_zip_code']


In [46]:
small['geolocation'].isnull().sum()

48379

In [47]:
no_dup = small.drop_duplicates(subset = ['federal_provider_number'])

In [48]:
no_dup['geolocation'].isnull().sum()

1186

In [49]:
# Fill location errors with 0s
# no_dup['geolocation']=no_dup['geolocation'].fillna(0)

In [50]:
no_dup.tail()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,geolocation,full_address
625387,676416,BRIGHTPOINTE AT LYTLE LAKE,1201 CLARKS DR,ABILENE,TX,79602,Taylor,"{'type': 'Point', 'coordinates': [-99.704298, ...","1201 CLARKS DR, ABILENE, TX, 79602"
625389,676419,LA HACIENDA DE PAZ REHABILITATION AND CARE CENTER,3333 BOB ROGERS DR,EAGLE PASS,TX,78852,Maverick,"{'type': 'Point', 'coordinates': [-100.457543,...","3333 BOB ROGERS DR, EAGLE PASS, TX, 78852"
625449,676477,WILLOW CREEK HEALTHCARE CENTRE,501 YATES STREET,MOUNT VERNON,TX,75457,Franklin,"{'type': 'Point', 'coordinates': [-95.228708, ...","501 YATES STREET, MOUNT VERNON, TX, 75457"
625527,676479,RICHARD A. ANDERSON (STATE OF TEXAS VETERANS LAND,14041 COTTINGHAM ROAD,HOUSTON,TX,77048,Harris,NaN,"14041 COTTINGHAM ROAD, HOUSTON, TX, 77048"
625557,686123,KENDALL LAKES HEALTH AND REHABILITATION CENTER,5280 SW 157 AVENUE,MIAMI,FL,33185,Miami-Dade,NaN,"5280 SW 157 AVENUE, MIAMI, FL, 33185"


In [51]:
no_dup = no_dup.reset_index()
no_dup.drop(columns='index', axis=1, inplace=True)

In [52]:
def fill_geo(df):
    for i in range(len(df)):
        location = geolocator.geocode(df['full_address'][i])
        if location==None:
            df['geolocation']='error'
        else:
            df['geolocation'][i]=[location.latitude, location.longitude]
    return df

In [53]:
new_df = fill_geo(no_dup)

In [53]:
location = geolocator.geocode('501 YATES STREET, MOUNT VERNON, TX, 75457')
print(location.latitude, location.longitude)

33.184225 -95.229151


In [41]:
# def geo_column_edit(dictionary):
#     if dictionary==0:
#         pass
#     else:
#         values = dictionary.values()
#         values_list = list(values)
#         return [values_list[1][1], values_list[1][0]]

In [13]:
# no_dup['geolocation']=no_dup['geolocation'].apply(geo_column_edit)

<ipython-input-13-9e272aab63bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_dup['geolocation']=no_dup['geolocation'].apply(geo_column_edit)


In [14]:
no_dup.head()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,geolocation,full_address
0,045277,ARKANSAS CONVALESCENT CENTER,6301 SOUTH HAZEL STREET,PINE BLUFF,AR,71603,Jefferson,None,"6301 SOUTH HAZEL STREET, PINE BLUFF, AR, 71603"
1,055516,WINDSOR CHICO CARE CENTER,188 COHASSET LANE,CHICO,CA,95926,Butte,"[39.750127, -121.852568]","188 COHASSET LANE, CHICO, CA, 95926"
2,056023,AVALON VILLA CARE CENTER,12029 AVALON BLVD,LOS ANGELES,CA,90061,Los Angeles,None,"12029 AVALON BLVD, LOS ANGELES, CA, 90061"
3,056317,GLENOAKS CONVALESCENT HOSPITAL,409 W. GLENOAKS BLVD.,GLENDALE,CA,91202,Los Angeles,"[34.159818, -118.26193000000002]","409 W. GLENOAKS BLVD., GLENDALE, CA, 91202"
4,05E119,TUSTIN CARE CENTER,1051 BRYAN AVENUE,TUSTIN,CA,92780,Orange,"[33.743106, -117.81555699999998]","1051 BRYAN AVENUE, TUSTIN, CA, 92780"


In [54]:
# NOTE: 1,199 have bad addresses. 
no_dup[no_dup['geolocation'].isnull()].shape

(0, 9)

In [55]:
no_dup.shape

(15407, 9)

In [57]:
new_df.to_csv('nhs_descriptors.csv')

## ALL TRIAL CODE GOES BELOW HERE

In [39]:
location = geolocator.geocode('409 W. GLENOAKS BLVD., GLENDALE, CA, 91202')
print(location.latitude, location.longitude)

34.159914072873924 -118.2624365591205


In [58]:
# location = geolocator.geocode("175 5th Avenue NYC")
# print((location.latitude, location.longitude))

(40.741059199999995, -73.98964162240998)


In [19]:
nulls = no_dup[no_dup['geolocation'].isnull()]

In [22]:
nulls['geolocation']

0             PINE BLUFF, AR, 71603
2            LOS ANGELES, CA, 90061
5                   YUMA, CO, 80759
6                  EAGLE, CO, 81631
15        ATLANTIC BEACH, FL, 32233
                    ...            
625302           DIMMITT, TX, 79027
625334           CYPRESS, TX, 77429
625365            IRVING, TX, 75038
625527           HOUSTON, TX, 77048
625557             MIAMI, FL, 33185
Name: geolocation, Length: 1186, dtype: object

In [21]:
nulls['geolocation']= nulls['provider_city'] + ', ' +nulls['provider_state'] + ', ' + nulls['provider_zip_code']

<ipython-input-21-ed354414b01e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nulls['geolocation']= nulls['provider_city'] + ', ' +nulls['provider_state'] + ', ' + nulls['provider_zip_code']


In [23]:
def geo_null(address):
    location = geolocator.geocode(address)
    return [location.latitude, location.longitude]

In [ ]:
400:500

In [33]:
nulls['geolocation'].iloc[450:500] = nulls['geolocation'].iloc[450:500].apply(geo_null)

In [26]:
nulls['geolocation'].iloc[200:300]

100238     [30.9038252, -84.5754768]
104705     [39.7990175, -89.6439575]
105669      [42.128704, -87.8265089]
107603    [-32.7761075, -71.2007014]
107860      [40.797821, -85.8205411]
                     ...            
161901     [43.1810844, -95.8561126]
162594     [38.2319537, -97.7733809]
163435     [37.2397486, -96.9955919]
163961     [30.4943669, -92.4176324]
166540     [38.2856247, -85.8241312]
Name: geolocation, Length: 100, dtype: object

In [31]:
nulls['geolocation'].iloc[600:]

325239                  [41.2134073, -98.4580649]
325294                 [37.6126634, -114.5142613]
325307                 [42.6867264, -103.4133215]
325332                  [41.4486334, -97.7307364]
325333                  [38.5246441, -118.624578]
                           ...                   
625302                 [34.5488229, -102.3153053]
625334                  [29.9691116, -95.6971686]
625365                  [32.8295183, -96.9442177]
625527                  [29.7589382, -95.3676974]
625557    [25.72919456538001, -80.44483666022803]
Name: geolocation, Length: 586, dtype: object

In [ ]:
nulls_z = nulls[['federal_provider_number', 'geolocation']]

In [49]:
new = no_dup.merge(nulls_z, on = 'federal_provider_number')

In [50]:
new.drop(columns = ['geolocation_x'], axis=1, inplace=True)

In [51]:
new.head()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,county,full_address,geolocation_y
0,035242,CHINLE NURSING HOME,HIGHWAY 191 & HOSPITAL ROAD,CHINLE,AZ,86503,Apache,"HIGHWAY 191 & HOSPITAL ROAD, CHINLE, AZ, 86503","[36.16763243080752, -109.68806973451998]"
1,065309,WASHINGTON COUNTY NURSING HOME,676 W GREENHOUSE DR,AKRON,CO,80720,Washington,"676 W GREENHOUSE DR, AKRON, CO, 80720","[49.82296369811355, 24.514349484476256]"
2,115275,"A.G. RHODES HOME, INC, THE","350 BOULVARD, S.E.",ATLANTA,GA,30312,Fulton,"350 BOULVARD, S.E., ATLANTA, GA, 30312","[33.74310470677914, -84.37445028551396]"
3,145323,CARRIER MILLS NSG & REHAB CTR,6789 US RT 45,CARRIER MILLS,IL,62917,Saline,"6789 US RT 45, CARRIER MILLS, IL, 62917","[37.682745708232396, -88.62537025784138]"
4,145424,LANDMARK OF RICHTON PARK REHAB & NSG CTR,22660 SOUTH CICERO AVENUE,RICHTON PARK,IL,60471,Cook,"22660 SOUTH CICERO AVENUE, RICHTON PARK, IL, 6...","[41.48200539573889, -87.72284133679652]"
